## Alternative Least Squares 

The above method is used here to find similar fighters to a particular fighter based on a paritcular feature (in this case I chose avg takedown percentages). This method will also be used to recommend fighters based on that same feature. Blue corner fighters were used here (B_fighters). However, red corner fighters(R_fighters) can also be used in this method  

In [93]:
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random
from sklearn.preprocessing import MinMaxScaler
import implicit

#load dataset. Blue corner fighters (B_fighter) is used here but is also applicable to fighters in the red corner as well ("R_fighters")
raw_data = pd.read_csv('finalView.csv')

# Drop NaN columns
data_new = raw_data.dropna()
data_new= data_new.copy()

#Convert fighter name and weight class to a category dtype  
data_new['weight_class_name'] = data_new["weight_class"].astype("category").cat.codes
data_new['B_fighter_name'] = data_new["B_fighter"].astype("category").cat.codes

##Enter fighter feature for recommendation. In this case i chose takedown percentages
fighter_feature=data_new["B_avg_TD_pct"]

#createing two matrices, one for finding similar fighters and one for best fighter recommendations in each weight_class
sparse_item_weight_class_name = sparse.csr_matrix((fighter_feature.astype(float), (data_new['B_fighter_name'], data_new['weight_class_name'])))
sparse_item_B_fighter_name = sparse.csr_matrix((fighter_feature.astype(float), (data_new['weight_class_name'], data_new['B_fighter_name'])))

In [94]:
# Initialize the als model and fit it using the sparse item feature-fighter matrix
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)
alpha_val = 40

# Calculate the data confidence by multiplying it by our alpha value.
data_conf = (sparse_item_weight_class_name * alpha_val).astype('double')

#Fit the model
model.fit(data_conf)

In [95]:
##get similar fighters based on the particular feature set above
def get_similar_fighters(fighter_name):
    #convert fighter name to index
    list1=data_new.index[data_new['B_fighter'] == fighter_name].tolist()
    fighter_id = list1[0] 
    n_similar = 10
    
    # Use implicit to get similar fighters.
    similar = model.similar_items(fighter_id, n_similar)
    #Show names of similar fighters
    for item in similar:
        idx, score = item
        print (data_new.B_fighter.loc[data_new.B_fighter_name == idx].iloc[0])

In [96]:
get_similar_fighters("Tony Ferguson")

Forrest Petz
Warlley Alves
Mike Swick
Greg Soto
Andre Pederneiras
Adlan Amagov
Jake Ellenberger
Li Jingliang
Chad Laprise
Erick Silva


In [102]:
get_similar_fighters("Amanda Nunes")

Valerie Letourneau
Marina Rodriguez
Weili Zhang
Jessica Aguilar
Bec Rawlings
Amanda Cooper
Sarah Frota
Rose Namajunas
Tecia Torres
Pearl Gonzalez


In [97]:
#Get best fighter recommendations based on selected feature above
def best_fighter_recommendations(weight_class_iter, sparse_fighter_content, fighter_vecs, weight_class_vecs, num_contents=10):
    
    # Get the array scores from the sparse weight class matrix
    fighter_array = sparse_item_B_fighter_name[weight_class_id,:].toarray()
    # Gives a new shape to an array without changing its data.
    fighter_array= fighter_array.reshape(-1) + 1
    
    # Mark visited items in array to 0
    fighter_array[fighter_array > 1] = 0
    
    # Get dot product of fighter vector and all weight class vectors
    recommendation_vector = fighter_vecs[weight_class_id,:].dot(weight_class_vecs.T).toarray()
    
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    recommendation_vector_scaled = min_max.fit_transform(recommendation_vector.reshape(-1,1))[:,0]
    # Scaling already visted items in array
    recommendation_vector_final = fighter_array * recommendation_vector_scaled
    # Sort the indices of the weight class into order of best recommendations
    weight_class_iter = np.argsort(recommendation_vector_final)[::-1][:num_contents]
    
    #Empty list to store fighters and scores
    fighters = []
    scores = []

    for iter in weight_class_iter:
        # Append fighters and scores to the list
        fighters.append(data_new.B_fighter.loc[data_new.B_fighter_name == iter].iloc[0])
        scores.append(recommendation_vector_final[iter])

    recommendations = pd.DataFrame({'Fighter': fighters, 'score': scores})

    return recommendations



In [101]:
# Get the fighter and weight class vectors converting them to csr matrices
fighter_vecs = sparse.csr_matrix(model.user_factors)
weight_class_vecs = sparse.csr_matrix(model.item_factors)

# Create best recommendations based on feature selected above. Set to 0 for best recommendations 
weight_class_id = 0

#Get recommendations
recommendations = best_fighter_recommendations(weight_class_id, sparse_item_B_fighter_name, fighter_vecs, weight_class_vecs)

print(recommendations)

            Fighter     score
0       Khalid Taha  0.066064
1     Ricardo Ramos  0.049482
2   Marcos Vinicius  0.046504
3    Jared Papazian  0.038212
4        Hugo Viana  0.033753
5    Johnny Eduardo  0.031349
6  Byron Bloodworth  0.030805
7      Damacio Page  0.028202
8   Chris Gutierrez  0.026209
9       Song Yadong  0.025825


References

https://medium.com/radon-dev/als-implicit-collaborative-filtering-5ed653ba39fe

https://towardsdatascience.com/building-a-collaborative-filtering-recommender-system-with-clickstream-data-dffc86c8c65